In [2]:
import numpy as np
import pandas as pd 

In [3]:
# import condition 
import os

# Extension of files needed:
ext = '.csv'
def import_data(ext, dir_path):
    # Conditions 
    condition = []
    
    # Loop through files: 
    for (dirpath, dirnames, filenames) in os.walk(dir_path):

        # Loop through Annotations:
        for i in range(len(filenames)):
            # Open files of interest:
            if filenames[i].startswith('.'):
                continue
            elif filenames[i].endswith(ext):
                # Read CSV
                current_activity = pd.read_csv(os.path.join(dirpath, filenames[i]))['activity']
                # Convert to numpy + Append to conditions 
                condition.append(current_activity.to_list())
    return condition

In [4]:
# Directory of interest:
condition_path = 'condition/'
control_path = 'control/'

control = import_data(ext, control_path)
control_label = [False]*len(control)
condition = import_data(ext, condition_path)
condition_label = [True]*len(condition)

In [5]:
inputData = control+condition
outputData = control_label+condition_label

In [6]:
from keras.utils import to_categorical


longest = max([len(x) for x in inputData])

inputData = np.array([[0]*(longest-len(x)) + x for x in inputData])
outputData = np.array(to_categorical(outputData))


print(len(inputData))
print(len(outputData))

55
55


In [7]:
from sklearn.model_selection import train_test_split


# Split data into training:
X_train, X_test, y_train, y_test = train_test_split(inputData, outputData,
                                                    test_size=0.25,
                                                    random_state=42)


In [8]:

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


print(X_train.shape)

(41, 65407, 1)


In [ ]:
from keras.layers.core import Dense, Dropout
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Flatten
import keras

classes = 1

model_name = 'stock_price_CNN'

model = Sequential()
model.add(Conv1D(256, kernel_size=3,
                 activation='relu',
                 input_shape=(X_train.shape[1], X_train.shape[2] )))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=50,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 41 samples, validate on 14 samples
Epoch 1/5
41/41 [==============================] - 183s 4s/step - loss: 6.8524 - acc: 0.4878 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 2/5
41/41 [==============================] - 263s 6s/step - loss: 5.8969 - acc: 0.6341 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 3/5
41/41 [==============================] - 236s 6s/step - loss: 3.5381 - acc: 0.7805 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 4/5
41/41 [==============================] - 262s 6s/step - loss: 5.8969 - acc: 0.6341 - val_loss: 4.6052 - val_acc: 0.7143
Epoch 5/5


In [ ]:

from keras.applications import Xception
from keras.layers import Dense, Flatten, Input
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential


def convolve(input_layer, conv_depth, kernel_size, pool_size):
    """ Perform 3 convolutions + down pool"""

    conv_1 = Conv2D(conv_depth, (kernel_size, kernel_size),
                    padding='same', activation='relu')(input_layer)
    conv_2 = Conv2D(conv_depth, (kernel_size, kernel_size),
                    padding='same', activation='relu')(conv_1)
    conv_3 = Conv2D(conv_depth, (kernel_size, kernel_size),
                    padding='same', activation='relu')(conv_2)

    return MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)



def build_network(inp_shape, num_classes, optimizer, loss_func, metrics):
    """ Main CNN to predict flat or not_flat  """

    model = Sequential()
    model.add(Xception(weights='imagenet', input_shape=inp_shape,
                       classes=num_classes, include_top=False))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=loss_func, optimizer=optimizer, metrics=metrics)

    # Output summary:
    print(model.summary())

    return model